모델의 기본적인 구조는 입력차원을 제외하면 이전과 같은 형태, 추출 특성을 바꿔서 실험

In [ ]:
#mfcc와 chroma만 추출해 보기로

def extract_mfcc_feature(file_name):
    
    y, sr = librosa.load(file_name, sr=16000)
    
    #spectrogram 구현
    spectrogram = librosa.stft(y, n_fft=512, hop_length= 256) #
    #   일반적으로 n_fft는 256, 512, 1024, 2048과 같은 2의 거듭제곱의 값 사용. 
    #   일반적으로 음성 신호의 경우 20ms에서 40ms 사이의 길이의 윈도우를 사용하는 것이 일반적
    #   대략 16000Hz의 샘플링 주파수를 갖는 경우, n_fft는 대략 320(= 16000 * 0.02)과 640(= 44100 * 0.04) 사이가 적절
    #   즉 이 경우 512가 가장 적절
    #   hop-length는 n_fft의 절반으로 설정하는 것이 일반적

    spectrogram = np.abs(spectrogram) #shape=(1025, 150)
    
    #Mel-spectrogram 구현
    power_spectrogram = spectrogram**2
    mel = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr)
    mel = librosa.power_to_db(mel)
    
    #mfcc 구현
    mfccs = librosa.feature.mfcc(S = mel, n_mfcc=200)
    mfccs = pad_sequences(mfccs, maxlen=200, padding='post',dtype='float32')
#     mfccs = normalize(mfccs,axis=1)
    mfccs = pad_sequences(mfccs.T, maxlen=60, padding='post',dtype='float32')
    
    #Chroma 구현
    stft = np.abs(librosa.stft(y))
    chroma_stft = librosa.feature.chroma_stft(S=stft,hop_length=512)
    chroma_stft = pad_sequences(chroma_stft, maxlen=200, padding='post',dtype='float32')
#     chroma_stft = normalize(chroma_stft,axis=1)
    chroma_stft = pad_sequences(chroma_stft.T, maxlen=60, padding='post',dtype='float32')
    
    #합치기
    
    features = np.hstack((mfccs,chroma_stft))
    features = np.pad(features,((0,0),(2,2)),mode='constant')
    features = scale(features, axis=1)
    
    return features

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten

def second_model(x_train):
    model=Sequential()
    model.add(InputLayer(shape=(400,64)))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #model.summary()
    return model


In [ ]:
def load_data(data_info, isTrain=True):
    if isTrain:
        X = []
#         file_list = data_info['wav_id']
        file_list = pd.DataFrame(os.listdir(test_path)).iloc[:,0] #테스트용

#         audio_path = (train_audio_path) 
        audio_path = test_path #테스트용
#         for file_name, emotion in tqdm(zip(file_list, emotion_list)):
        for file_name in tqdm(file_list):
        
#             features = extract_mfcc_feature(os.path.join(audio_path, f'{file_name}.wav'))
            features = extract_mfcc_feature(os.path.join(audio_path, f'{file_name}')) #테스트용
            
            X.append(features)
#         train_data= np.array(X).reshape(-1,400,64) 
            
        return X #, np.array(train_label)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#스케일링 - 데이터를 평균 0, 표준편차 1로 스케일을 축소

y = train_csv.iloc[:,-7:]

# scaler = StandardScaler()

   
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0,  test_size = 0.3, stratify = y)

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_train)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

model = second_model(X_train)
history=model.fit(X_train, y_train, batch_size=40, epochs=50, validation_data=(X_test, y_test), callbacks=[rlrp,mcp])

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: ",test_acc)

#결과 tes_acc는 약 42%가 최대